In [2]:
import ibis
from pathlib import Path
import ibis_widget
import election_data as ed
from ibis.expr import types as ir

ibis_widget.install()
ibis.options.interactive = True

from ibis import _

In [3]:
# see https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/
# There are other races available, such as primaries.
urls = {
    2017: "https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/2017/2017%20November%20General.csv",
    2019: "https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/2019/2019%20November%20General.csv",
    2021: "https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/2021/2021%20November%20General%20.csv",
    # 2023 isn't available yet :( https://apps.elections.virginia.gov/SBE_CSV/ELECTIONS/ELECTIONRESULTS/2023/
}
paths = {year: Path(f"../data/va/{year}.csv") for year in urls}
for year, url in urls.items():
    path = paths[year]
    path.parent.mkdir(parents=True, exist_ok=True)
    if not path.exists():
        print(f"Downloading {year} results to {path}")
        !curl -o {path} {url}

In [31]:
raws = [ibis.read_csv(paths[year]).mutate(year=year) for year in urls]
raw = ibis.union(*raws)
raw.widget()

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ __row_id ┃ CandidateUid                           ┃ FirstName ┃ MiddleName ┃ LastName       ┃ Suffix ┃ TOTAL_VOTES ┃ Party       ┃ WriteInVote ┃ LocalityUid                            ┃ LocalityCode ┃ LocalityName    ┃ PrecinctUid ┃ PrecinctName                     ┃ DistrictUid                            ┃ DistrictType       ┃ DistrictName ┃ OfficeUid                              ┃ OfficeTitle               ┃ ElectionUid                            ┃ ElectionType ┃ El

In [33]:
def make_county_fips_lookup() -> ibis.Table:
    t = ibis.read_parquet("../data/cleaned.parquet")
    return t.select(
        "state_po",
        "county_name",
        "county_fips",
    ).distinct()


def add_county_fips(t: ibis.Table) -> ibis.Table:
    lookup = make_county_fips_lookup()
    return t.left_join(lookup, ["state_po", "county_name"]).drop(
        "state_po_right", "county_name_right"
    )


t = raw
assert (t.ElectionType == "General").all().execute()
t = t.drop("ElectionType")
t = t.drop("ElectionName")

t = t.drop(
    "CandidateUid",
    "LocalityUid",
    "LocalityCode",
    "PrecinctUid",
    "DistrictUid",
    "OfficeUid",
    "ElectionUid",
)
t = t.mutate(
    election_date=_.ElectionDate.date(),
    state_po=ibis.literal("VA"),
    county_name=_.LocalityName.replace(" & ", " and "),
    candidate=(_.FirstName + " " + _.MiddleName + " " + _.LastName + " " + _.Suffix)
    .re_replace(r"\s+", " ")
    .strip()
    .upper(),
).drop("ElectionDate", "LocalityName", "FirstName", "MiddleName", "LastName", "Suffix")

assert (t.election_date.nunique() == len(urls)).execute()

t = add_county_fips(t)
t.widget()

┏━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ __row_id ┃ TOTAL_VOTES ┃ Party       ┃ WriteInVote ┃ PrecinctName                      ┃ DistrictType       ┃ DistrictName ┃ OfficeTitle                                                     ┃ year  ┃ election_date ┃ state_po ┃ county_name       ┃ candidate ┃ county_fips ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64    │ int64       │ string      │ int64       │ string                            │ string             │ string       │ string                                             

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ ElectionType ┃ CountStar() ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string       │ int64       │
├──────────────┼─────────────┤
│ General      │       49040 │
└──────────────┴─────────────┘

In [16]:
assert (t.filter(_.WriteInVote != 0).LastName == "WRITE IN VOTES").all().execute()
assert (t.filter(_.WriteInVote == 0).LastName != "WRITE IN VOTES").all().execute()
assert (t.filter(_.LastName == "WRITE IN VOTES").WriteInVote == 1).all().execute()
assert (t.filter(_.LastName != "WRITE IN VOTES").WriteInVote == 0).all().execute()
t = t.mutate(
    writein=_.WriteInVote == 1,
    candidate=ibis.case().when(_.WriteInVote == 1, "WRITEIN").else_(),
).drop("WriteInVote")

In [23]:
t.widget()

┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ __row_id ┃ FirstName ┃ MiddleName ┃ LastName ┃ Suffix ┃ TOTAL_VOTES ┃ Party       ┃ WriteInVote ┃ PrecinctName                      ┃ DistrictType       ┃ DistrictName ┃ OfficeTitle               ┃ ElectionType ┃ ElectionName          ┃ election_date ┃ state_po ┃ county_name       ┃ state_po_right ┃ county_name_right ┃ county_fips ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━

In [7]:
t.filter(_.county_fips.isnull())

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ CandidateUid ┃ FirstName ┃ MiddleName ┃ LastName       ┃ Suffix ┃ TOTAL_VOTES ┃ Party  ┃ WriteInVote ┃ LocalityUid                            ┃ LocalityCode ┃ LocalityName  ┃ PrecinctUid ┃ PrecinctName                     ┃ DistrictUid                            ┃ DistrictType       ┃ DistrictName           ┃ OfficeUid                              ┃ OfficeTitle                 ┃ ElectionUid                            ┃ ElectionType ┃ ElectionDate        ┃ ElectionName          ┃ state_po ┃ county_name   ┃ state_po_right ┃ county_name_right ┃ county_fips ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string       │ string    │ string     │ string         │ string │ int64       │ string │ int64       │ string                                 │ string       │ string        │ string      │ string                           │ string                                 │ string             │ string                 │ string                                 │ string                      │ string                                 │ string       │ timestamp(6)        │ string                │ string   │ string        │ string         │ string            │ string      │
├──────────────┼───────────┼────────────┼────────────────┼────────┼─────────────┼────────┼─────────────┼────────────────────────────────────────┼──────────────┼───────────────┼─────────────┼──────────────────────────────────┼────────────────────────────────────────┼────────────────────┼────────────────────────┼────────────────────────────────────────┼─────────────────────────────┼────────────────────────────────────────┼──────────────┼─────────────────────┼───────────────────────┼──────────┼───────────────┼────────────────┼───────────────────┼─────────────┤
│ NULL         │ NULL      │ NULL       │ WRITE IN VOTES │ NULL   │           0 │ NULL   │           1 │ {49789A69-BC49-4149-9039-E6BC1FF275C0} │ 109          │ LOUISA COUNTY │ NULL        │ # AB - Central Absentee Precinct │ {3393B208-7DE9-4E12-AE6E-15FFDC032EE8} │ Congressional      │ 07                     │ {C9719525-54D6-4AD4-9F6B-72068D53BABE} │ Lieutenant Governor         │ {C0DBB229-5744-4187-9DAB-A85AEEFED160} │ General      │ 2017-11-07 00:00:00 │ 2017 November General │ VA       │ LOUISA COUNTY │ NULL           │ NULL              │ NULL        │
│ NULL         │ NULL      │ NULL       │ WRITE IN VOTES │ NULL   │           2 │ NULL   │           1 │ {49789A69-BC49-4149-9039-E6BC1FF275C0} │ 109          │ LOUISA COUNTY │ NULL        │ # AB - Central Absentee Precinct │ {05C61C07-81AF-42C6-9BB2-3AE5E7832D3B} │ Election           │ MOUNTAIN ROAD DISTRICT │ {BB1A6EAE-27BC-4D1E-903A-3D01E8DEA685} │ Member Board of Supervisors │ {C0DBB229-5744-4187-9DAB-A85AEEFED160} │ General      │ 2017-11-07 00:00:00 │ 2017 November General │ VA       │ LOUISA COUNTY │ NULL           │ NULL              │ NULL        │
│ NULL         │ NULL      │ NULL       │ WRITE IN V

In [ ]:
cleaned = ed.make_results(
    t,
    year=2017,
    date="2017-11-07",
    state_po="VA",
    county_name=_.LocalityName.replace(" & ", " and "),
)

In [22]:
gold = ibis.read_parquet("../data/cleaned.parquet")
gold.filter(
    True,
    _.state_po == "VA",
    _.candidate.length() > 10,
).widget()

┏━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ __row_id ┃ year  ┃ date       ┃ state_po ┃ county_name ┃ county_fips ┃ jurisdiction_name ┃ jurisdiction_fips ┃ district ┃ office      ┃ magnitude ┃ special ┃ stage  ┃ precinct                      ┃ writein ┃ candidate         ┃ party_detailed ┃ party_simplified ┃ mode         ┃ votes  ┃ readme_check ┃
┡━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━┩
│ int64    │ int16 │ date       │ string   │ string      │ string      │ string   

In [4]:
# ported from https://github.com/MEDSL/replication-scripts/blob/634832bc7c2df2c0a2238bc16781d07d581d3a3a/va2018.py


# Fill NA values with an empty string
t = t.fill_null("")

# Replace specific county name
t = t.mutate(
    county_name=ibis.case()
    .when(t.county_name == "KING & QUEEN", "KING AND QUEEN")
    .else_(t.county_name)
    .end()
)

# Load and merge FIPS codes
fips = con.table("../../../help-files/county-fips-codes.csv")
fips = fips.mutate(state=fips.state.upper())
t = t.join(fips, ["state", "county_name"], how="left")
t = t.mutate(county_fips=t.county_fips.cast("string").zfill(5))

# Load jurisdiction FIPS and merge based on conditions
juris_fips = con.table("../../../help-files/jurisdiction-fips-codes.csv")
juris_fips = juris_fips.mutate(state=juris_fips.state.upper())

states_w_juris = (
    juris_fips.filter(juris_fips.jurisdiction_fips.length() > 5)
    .state.distinct()
    .to_list()
)
if t.state.distinct().execute()[0] not in states_w_juris:
    t = t.mutate(jurisdiction_fips=t.county_fips, jurisdiction_name=t.county_name)
else:
    juris_fips = juris_fips.mutate(
        county_fips=juris_fips.jurisdiction_fips.str.zfill(10).substr(0, 5)
    )
    t = t.join(juris_fips, ["state", "county_fips", "jurisdiction_name"], how="left")
    t = t.mutate(
        jurisdiction_fips=ibis.ifelse(
            t.jurisdiction_fips.isnull(),
            "Missing Jurisdiction FIPS",
            t.jurisdiction_fips,
        )
    )

# Pad and clean district codes
t = t.mutate(
    district=t.district.zfill(3).replace(
        {"00A": "A", "00B": "B", "00C": "C", "000": ""}
    )
)

# Set date and readme_check values
t = t.mutate(date="2018-11-06", readme_check="FALSE")

# Merge magnitudes data
mags = con.table("magnitudes.csv").fillna("")
t = t.join(mags, ["office", "jurisdiction_name", "district"], how="left")
t = t.mutate(magnitude=t.magnitude.fillna(1).cast("int64"))

# Write-in candidate replacements
t = t.mutate(
    candidate=t.candidate.replace("[WRITE-IN]", "WRITEIN", regex=False).replace(
        r"\.", "", regex=True
    )
)

# Apply specific conditions for certain office values
t = t.mutate(
    district=ibis.case()
    .when(t.office == "MEMBER BOARD OF SUPERVISORS AT LARGE", "AT-LARGE")
    .else_(t.district)
    .end(),
    office=ibis.case()
    .when(
        t.office == "MEMBER BOARD OF SUPERVISORS AT LARGE",
        "MEMBER BOARD OF SUPERVISORS",
    )
    .else_(t.office)
    .end(),
)

# Load amendments data and concatenate
ammendments = con.table("va-18-ammendments.csv", schema=official_dtypes)
t = t.union(ammendments)

# Replace boolean values with string equivalents and trim extra spaces
t = t.replace({True: "TRUE", False: "FALSE"})
t = t.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Export to CSV
t.to_csv(
    "2018-va-precinct-general-updated.csv", quoting=csv.QUOTE_NONNUMERIC, index=False
)


AttributeError: 'Table' object has no attribute 'fill_null'